---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [13]:
len(set(list(P1_Graphs[0].degree().values())))

29

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [18]:
def graph_identification():
    
    ans = []
    for G in P1_Graphs:
        degree = G.degree()
        unique = set(list(degree.values()))
        clustering = nx.average_clustering(G)
        if len(unique) > 10:
            ans.append('PA')
        elif clustering < 0.1:
            ans.append('SW_H')
        else:
            ans.append('SW_L')
    
    return ans

In [19]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [20]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [138]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, auc

def salary_predictions():
    
    # prepare dataframe
    df = pd.DataFrame(G.nodes(data=True), columns=['person', 'department'])
    
    df.index = df['person']
    df = df.drop(['person'],axis=1)
    
    df['managementsalary'] = df['department'].map(lambda x: x['ManagementSalary'])
    df['department'] = df['department'].map(lambda x: x['Department'])
    
    # add in features
    df['degree'] = pd.Series(G.degree())
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness_centrality'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))

    # graph is not connected, total 20 components while only one has more than 1 node
    # assign components    
    c = [c for c in nx.connected_components(G)]
    for i in range(len(c)):
        for j in range(df.shape[0]):
            if df.index[j] in c[i]:
                df.loc[df.index[j],'component'] = i
    
    # see whether component is center in largest component, only 2 periphery in largest component so not label periphery
    G1 = max(nx.connected_component_subgraphs(G), key = len)
    center = nx.center(G1)
    for i in range(df.shape[0]):
        if df.index[i] in center:
            df.loc[df.index[i],'center'] = 1
        else:
            df.loc[df.index[i],'center'] = 0
    
    # prepare test set
    X_test = df[df['managementsalary'].isnull()]
    X_test = X_test.drop(['managementsalary'], axis=1)
    
    # split feature and result
    X = df.dropna()
    y = X['managementsalary'].copy()
    X = X.drop(['managementsalary'], axis =1)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)
    
    feature = ['degree', 'clustering', 'degree_centrality',
               'closeness_centrality', 'betweeness_centrality', 'pr']

    #feature = ['department', 'degree', 'clustering', 'degree_centrality',
    #           'closeness_centrality', 'betweeness_centrality', 'pr', 'component']
        
    X_train_fea = X_train[feature]
    X_valid_fea = X_valid[feature]
    X_test_fea = X_test[feature]
    
    gbm = GradientBoostingClassifier(learning_rate=0.1, min_samples_leaf = 50, max_depth = 5, 
                                     min_samples_split = 5, n_estimators = 80,
                                     random_state=10, max_features='sqrt',subsample=0.8)
    gbm.fit(X_train_fea, y_train)
    prob = gbm.predict_proba(X_test_fea)
    m_prob = [i[1] for i in prob]
           
    return pd.Series(m_prob,X_test.index)

In [139]:
salary_predictions()

person
1       0.023811
2       0.749875
5       0.832905
8       0.381886
14      0.098533
18      0.051190
27      0.042276
30      0.409860
31      0.485163
34      0.058193
37      0.036815
40      0.024093
45      0.008590
54      0.457690
55      0.293528
60      0.240574
62      0.832905
65      0.832905
77      0.013880
79      0.036449
97      0.008144
101     0.003364
103     0.499426
108     0.019946
113     0.164163
122     0.001347
141     0.776721
142     0.832905
144     0.016359
145     0.419323
          ...   
913     0.002796
914     0.003761
915     0.000491
918     0.054366
923     0.013631
926     0.112351
931     0.001907
934     0.000287
939     0.000276
944     0.000491
945     0.013575
947     0.041946
950     0.068103
951     0.003623
953     0.000336
959     0.000490
962     0.000247
963     0.285832
968     0.029862
969     0.033284
974     0.044590
984     0.000377
987     0.049225
989     0.029862
991     0.029862
992     0.000890
994     0.000490
996    

In [142]:
#gbm0 = GradientBoostingClassifier(random_state=10)
#gbm0.fit(X_train_fea, y_train)

#y_valid_pred = gbm0.predict(X_valid_fea)
#y_valid_score = gbm0.decision_function(X_valid_fea)
#cv_score = cross_val_score(gbm0, X_train_fea, y_train, cv=5, scoring='roc_auc')
#fp, tp,_ = roc_curve(y_valid, y_valid_score)
#rocauc = auc(fp,tp)
#print("Accuracy score (training): {0:.3f}".format(gbm0.score(X_train_fea, y_train)))
#print("Accuracy score (validation): {0:.3f}".format(gbm0.score(X_valid_fea, y_valid)))
#print("CV Score : Mean - {} | Std - {} | Min - {} | Max - {}".format(np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
#print("Confusion Matrix:")
#print(confusion_matrix(y_valid, y_valid_pred))
#print("Classification Report")
#print(classification_report(y_valid, y_valid_pred))
#print("Area under ROC curve = {:0.2f}".format(rocauc))
#Accuracy score (training): 0.995
#Accuracy score (validation): 0.910
#CV Score : Mean - 0.950670794633643 | Std - 0.005732556999676571 | Min - 0.9456140350877192 | Max - 0.9603715170278637
#Confusion Matrix:
#[[158   1]
# [ 16  14]]
#Classification Report
#             precision    recall  f1-score   support

#        0.0       0.91      0.99      0.95       159
#        1.0       0.93      0.47      0.62        30

#avg / total       0.91      0.91      0.90       189

#Area under ROC curve = 0.96

In [140]:
#gbm1 = GradientBoostingClassifier(learning_rate=0.1, 
#                                  min_samples_leaf = 50, random_state=10, max_features='sqrt',subsample=0.8)
#param_grid = {'n_estimators':range(20,81,10),
#              'min_samples_split':range(5,12),
#              'max_depth':range(5,8)}
#grid_gbm1 = GridSearchCV(gbm1, param_grid=param_grid, scoring='roc_auc',n_jobs=4, cv=5)
#grid_gbm1.fit(X_train_fea,y_train)
#grid_gbm1.best_params_, grid_gbm1.best_score_

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [141]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [205]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, auc

def new_connections_predictions():
    
    def frame_converter(ls, column_name):
        idx = [(i[0], i[1]) for i in ls]
        return pd.DataFrame([i[2] for i in ls], index=idx, columns = column_name)
      
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']

    # prepare df    
    jac_co = frame_converter(ls=[c for c in nx.jaccard_coefficient(G)], column_name=['jaccard_coefficient'])
    pre_att = frame_converter(ls=[c for c in nx.preferential_attachment(G)], column_name=['preferential_attachment'])
    re_allo_idx = frame_converter(ls=[c for c in nx.resource_allocation_index(G)], column_name=['resource_allocation_index'])
    sound_hop = frame_converter(ls=[c for c in nx.cn_soundarajan_hopcroft(G)], column_name=['cn_soundarajan_hopcroft'])
    df = pd.concat([jac_co, pre_att, re_allo_idx, sound_hop], axis=1)
    df_2 = pd.merge(df, future_connections, how='left', left_index=True, right_index=True)
    
    # prepare train and test sets
    X = df_2[~pd.isnull(df_2['Future Connection'])]
    X_test = df_2[pd.isnull(df_2['Future Connection'])]
    y = X['Future Connection'].copy()
    
    # train test split
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)
    
    # model building
    feature = ['jaccard_coefficient','preferential_attachment','resource_allocation_index','cn_soundarajan_hopcroft']
    X_train_fea = X_train[feature]
    X_valid_fea = X_valid[feature]
    X_test_fea = X_test[feature]
    
    gbm0 = GradientBoostingClassifier(random_state=10)
    gbm0.fit(X_train_fea, y_train)
    
    prob = gbm0.predict_proba(X_test_fea)
    link_prob = [p[1] for p in prob]
    
    return pd.Series(link_prob, X_test_fea.index)

In [206]:
new_connections_predictions()

(0, 9)          0.038208
(0, 19)         0.071482
(0, 20)         0.216836
(0, 35)         0.017173
(0, 38)         0.013478
(0, 42)         0.845211
(0, 43)         0.015081
(0, 50)         0.013052
(0, 53)         0.524259
(0, 54)         0.040278
(0, 62)         0.351510
(0, 63)         0.066505
(0, 69)         0.029310
(0, 72)         0.012853
(0, 83)         0.181711
(0, 90)         0.030207
(0, 91)         0.015572
(0, 95)         0.112478
(0, 100)        0.030949
(0, 106)        0.990854
(0, 108)        0.017173
(0, 109)        0.012853
(0, 110)        0.012853
(0, 118)        0.013478
(0, 122)        0.028055
(0, 131)        0.040278
(0, 133)        0.706688
(0, 140)        0.072452
(0, 149)        0.147962
(0, 151)        0.013478
                  ...   
(988, 989)      0.012853
(988, 996)      0.012853
(988, 997)      0.061383
(988, 1000)     0.012853
(988, 1002)     0.012853
(989, 994)      0.012853
(989, 996)      0.012853
(989, 1003)     0.012853
(989, 1004)     0.012853


In [199]:
feature = ['jaccard_coefficient','preferential_attachment','resource_allocation_index','cn_soundarajan_hopcroft']

X_train_fea = X_train[feature]
X_valid_fea = X_valid[feature]
X_test_fea = X_test[feature]

In [201]:
#gbm0 = GradientBoostingClassifier(random_state=10)
#gbm0.fit(X_train_fea, y_train)

#y_valid_pred = gbm0.predict(X_valid_fea)
#y_valid_score = gbm0.decision_function(X_valid_fea)
#cv_score = cross_val_score(gbm0, X_train_fea, y_train, cv=5, scoring='roc_auc')
#fp, tp,_ = roc_curve(y_valid, y_valid_score)
#rocauc = auc(fp,tp)
#print("Accuracy score (training): {0:.3f}".format(gbm0.score(X_train_fea, y_train)))
#print("Accuracy score (validation): {0:.3f}".format(gbm0.score(X_valid_fea, y_valid)))
#print("CV Score : Mean - {} | Std - {} | Min - {} | Max - {}".format(np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
#print("Confusion Matrix:")
#print(confusion_matrix(y_valid, y_valid_pred))
#print("Classification Report")
#print(classification_report(y_valid, y_valid_pred))
#print("Area under ROC curve = {:0.2f}".format(rocauc))

#Accuracy score (training): 0.963
#Accuracy score (validation): 0.963
#CV Score : Mean - 0.9141766699699753 | Std - 0.003545358034564728 | Min - 0.9072222715373918 | Max - 0.9167552294830033
#Confusion Matrix:
#[[83606   687]
# [ 2732  4559]]
#Classification Report
#             precision    recall  f1-score   support

#        0.0       0.97      0.99      0.98     84293
#        1.0       0.87      0.63      0.73      7291

#avg / total       0.96      0.96      0.96     91584

#Area under ROC curve = 0.91

In [204]:
#I tried hyperparameter tuning but seems that's overloading the computer power, I just put my grid search algo here
#gbm1 = GradientBoostingClassifier(learning_rate=0.1, 
#                                  min_samples_leaf = 50, random_state=10, max_features='sqrt',subsample=0.8)
#param_grid = {'n_estimators':range(20,81,10),
#              'min_samples_split':range(2500,5000,500),
#              'max_depth':range(8,15)}
#grid_gbm1 = GridSearchCV(gbm1, param_grid=param_grid, scoring='roc_auc',n_jobs=4, cv=5)
#grid_gbm1.fit(X_train_fea,y_train)
#grid_gbm1.best_params_, grid_gbm1.best_score_